Автор блокнота: Олег Дудник

Мануал по SQL: http://swcarpentry.github.io/sql-novice-survey/10-prog/index.html

In [2]:
# Cell #1
import sqlite3
conn = sqlite3.connect('restaurant.db') # Создаем соединение с нашей базой данных

# Часть №1

#### Хотим таблицу Склад:

| Название ингредиента | Количество | Дата* |
| -------------------- |:----------:|:-----:|
| Брокколи             | 25         | 02.12 |
| Куриное филе         | 5          | 03.12 |
| Брокколи             | 30         | 04.12 |

*когда испортиться или когда привезли

#### Как ее правильно реализовать в реляционных базах данных?
###### Таблица Склад:  
   
| id | id ингредиента | Количество | Дата* |
| -- | -------------- |:----------:|:-----:|
| 1  | 5              | 25         | 1575314617 |
| 2  | 7              | 5          | 1575401017 |
| 3  | 5              | 30         | 1575487417 |

###### Таблица Ингредиенты:

| id ингредиента | Название |
| -- | -------------- |
| 5  | Брокколи       |
| 7  | Куриное филе   |

*unix-time

Почему именно так?

In [3]:
# Cell #2
import sys
print(sys.getsizeof(318))
print(sys.getsizeof('Брокколи'))
d = sys.getsizeof('Брокколи') - sys.getsizeof(318)
print(d)

28
90
62


Создадим таблицы в нашей БД:

In [4]:
# Cell #3
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
c = conn.cursor()

# Создаем таблицу ингредиентов
c.execute('''CREATE TABLE ingredients
             (id INTEGER PRIMARY KEY, name TEXT)''')

# Создаем таблицу для склада
c.execute('''CREATE TABLE warehouse
             (id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ingredient_id INTEGER, count INTEGER, date REAL)''')

# Save (commit) the changes
conn.commit()

Заполним таблицу ингредиентов:

In [5]:
# Cell #4
ingredients = ['Молоко', 'Яйцо', 'Мука', 'Разрыхлитель', 'Масло растительное', 'Сахар', 'Соль']

for i in map(lambda it: (it, ), ingredients):
    print(i)
    c.execute("INSERT INTO ingredients (name) VALUES (?)", i)

('Молоко',)
('Яйцо',)
('Мука',)
('Разрыхлитель',)
('Масло растительное',)
('Сахар',)
('Соль',)


Посмотрим на неё:

In [6]:
# Cell #5
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко'),
 (2, 'Яйцо'),
 (3, 'Мука'),
 (4, 'Разрыхлитель'),
 (5, 'Масло растительное'),
 (6, 'Сахар'),
 (7, 'Соль')]

In [7]:
# Cell #6
c.execute("SELECT name FROM ingredients")
results = c.fetchall()
results

[('Молоко',),
 ('Яйцо',),
 ('Мука',),
 ('Разрыхлитель',),
 ('Масло растительное',),
 ('Сахар',),
 ('Соль',)]

In [8]:
# Cell #7
c.execute("SELECT * FROM ingredients WHERE id=5")
print(c.fetchall())

c.execute("SELECT * FROM ingredients WHERE name='Соль'")
print(c.fetchall())

[(5, 'Масло растительное')]
[(7, 'Соль')]


Посчитаем количество строк этой таблицы:

In [9]:
# Cell #8
c.execute("SELECT COUNT(*) FROM ingredients")
print(c.fetchall())

c.execute("SELECT SUM(id) FROM ingredients")
print(c.fetchall())

[(7,)]
[(28,)]


Создадим случайные данные для Склада и запишем их в БД:

In [10]:
# Cell #9
c.execute("SELECT id FROM ingredients")
results = c.fetchall()
print(results)
ids = list(map(lambda it: it[0], results))
print(ids)

[(1,), (2,), (3,), (4,), (5,), (6,), (7,)]
[1, 2, 3, 4, 5, 6, 7]


In [11]:
# Cell #10
import random
import datetime

for i in range(1, 28):
    count = random.randint(10, 100)
    date = datetime.datetime(2019, 11, i, hour=random.randint(12, 18), minute=random.randint(0, 59), second=0, microsecond=0, tzinfo=None)
    ingredient = random.choice(ids)
    c.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", (ingredient, count, date.timestamp()))

In [12]:
# Cell #11
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

[(1, 7, 80, 1572621120.0),
 (2, 3, 99, 1572698520.0),
 (3, 1, 89, 1572775920.0),
 (4, 2, 50, 1572868260.0),
 (5, 3, 73, 1572949860.0),
 (6, 1, 98, 1573032000.0),
 (7, 2, 23, 1573127220.0),
 (8, 6, 84, 1573224420.0),
 (9, 1, 12, 1573290180.0),
 (10, 4, 31, 1573387260.0),
 (11, 2, 19, 1573468560.0),
 (12, 5, 32, 1573562040.0),
 (13, 6, 33, 1573650480.0),
 (14, 4, 20, 1573734480.0),
 (15, 5, 64, 1573828320.0),
 (16, 5, 11, 1573913400.0),
 (17, 4, 94, 1573992960.0),
 (18, 5, 60, 1574071200.0),
 (19, 2, 61, 1574162040.0),
 (20, 4, 43, 1574254620.0),
 (21, 4, 32, 1574345760.0),
 (22, 5, 25, 1574418900.0),
 (23, 6, 12, 1574513580.0),
 (24, 6, 40, 1574598000.0),
 (25, 4, 41, 1574686680.0),
 (26, 1, 30, 1574765220.0),
 (27, 6, 70, 1574859780.0)]

Приведем все в нормальный вид:

In [13]:
# Cell #12
def prettyWarehouse(cursor):
    cursor.execute("""SELECT name, count, date FROM warehouse
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id""")
    return cursor.fetchall()

prettyWarehouse(c)

[('Соль', 80, 1572621120.0),
 ('Мука', 99, 1572698520.0),
 ('Молоко', 89, 1572775920.0),
 ('Яйцо', 50, 1572868260.0),
 ('Мука', 73, 1572949860.0),
 ('Молоко', 98, 1573032000.0),
 ('Яйцо', 23, 1573127220.0),
 ('Сахар', 84, 1573224420.0),
 ('Молоко', 12, 1573290180.0),
 ('Разрыхлитель', 31, 1573387260.0),
 ('Яйцо', 19, 1573468560.0),
 ('Масло растительное', 32, 1573562040.0),
 ('Сахар', 33, 1573650480.0),
 ('Разрыхлитель', 20, 1573734480.0),
 ('Масло растительное', 64, 1573828320.0),
 ('Масло растительное', 11, 1573913400.0),
 ('Разрыхлитель', 94, 1573992960.0),
 ('Масло растительное', 60, 1574071200.0),
 ('Яйцо', 61, 1574162040.0),
 ('Разрыхлитель', 43, 1574254620.0),
 ('Разрыхлитель', 32, 1574345760.0),
 ('Масло растительное', 25, 1574418900.0),
 ('Сахар', 12, 1574513580.0),
 ('Сахар', 40, 1574598000.0),
 ('Разрыхлитель', 41, 1574686680.0),
 ('Молоко', 30, 1574765220.0),
 ('Сахар', 70, 1574859780.0)]

Посчитаем количество ингредиентов на складе:

In [14]:
# Cell #13
def ingredientsOnWarehouse(cursor):
    cursor.execute("""SELECT name, SUM(count) FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name""")
    return cursor.fetchall()

ingredientsOnWarehouse(c)

[('Масло растительное', 192),
 ('Молоко', 229),
 ('Мука', 172),
 ('Разрыхлитель', 261),
 ('Сахар', 239),
 ('Соль', 80),
 ('Яйцо', 153)]

Создадим функцию для поставки ингредиентов:

In [15]:
# Cell #14
c.execute("SELECT id FROM ingredients WHERE name=?", ('Мука',))
results = c.fetchall()
print(results)
ingredient_id = results[0][0]
ingredient_id

[(3,)]


3

In [16]:
# Cell #15
c.execute("SELECT id FROM ingredients WHERE name=?", ('Авокадо',))
results = c.fetchall()
print(results)
print(len(results))

[]
0


In [17]:
# Cell #16
def getIngredientId(cursor, name):
    cursor.execute("SELECT id FROM ingredients WHERE name=?", (name, ))
    results = cursor.fetchall()
    if(len(results) > 0):
        return results[0][0]
    else:
        cursor.execute("INSERT INTO ingredients (name) VALUES (?)", (name, ))
        return getIngredientId(cursor, name)

In [18]:
# Cell #17
import datetime
def supply(cursor, ingredient_name, count, date):
    ingredient_id = getIngredientId(cursor, ingredient_name)
    cursor.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", 
                       (ingredient_id, count, date.timestamp()))

In [19]:
# Cell #18
c.execute("DELETE FROM warehouse");
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

[]

In [20]:
# Cell #19
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко'),
 (2, 'Яйцо'),
 (3, 'Мука'),
 (4, 'Разрыхлитель'),
 (5, 'Масло растительное'),
 (6, 'Сахар'),
 (7, 'Соль')]

In [21]:
# Cell #20
supply(c, 'Мука', 10, datetime.datetime.now())
supply(c, 'Авокадо', 10, datetime.datetime.now())

In [22]:
# Cell #21
prettyWarehouse(c)

[('Мука', 10, 1583761968.367144), ('Авокадо', 10, 1583761968.367744)]

In [23]:
# Cell #22
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко'),
 (2, 'Яйцо'),
 (3, 'Мука'),
 (4, 'Разрыхлитель'),
 (5, 'Масло растительное'),
 (6, 'Сахар'),
 (7, 'Соль'),
 (8, 'Авокадо')]

# Часть №2

#### Хотим таблицу Блюд:
   
| id | Название            | Ингредиенты |
| -- | ------------------- |:-----------:|
| 1  | Панкейки            | ?           |
| 2  | Буррито             | ?           |
| 3  | Овощи по деревенски | ?           |


#### Как ее правильно реализовать в реляционных базах данных?

###### Таблица Блюд:

| id | Название            |
| -- | ------------------- |
| 1  | Панкейки            |
| 2  | Буррито             |
| 3  | Овощи по деревенски |

###### Таблица Ингредиентов:

| id ингредиента | Название |
| -- | -------------- |
| 5  | Брокколи       |
| 7  | Куриное филе   |
| 12 | Молоко         |

###### Таблица Блюд-Ингредиентов:

| id Блюда | id Ингредиента | Количество ингредиентов |
| -------- | -------- | -------- |
| 1        | 12       | 250 |
| 2        | 7       | 50 |
| 3        | 5       | 10 |

In [24]:
# Cell #23
c.execute("DELETE FROM ingredients");
c.execute("DELETE FROM warehouse");

In [25]:
# Cell #24
# Создаем таблицу блюд
c.execute('''CREATE TABLE dishes
             (id INTEGER PRIMARY KEY, name TEXT, recipe INTEGER)''')

# Создаем таблицу для блюд-ингредиетов
c.execute('''CREATE TABLE dishes_ingredients
             (dish_id INTEGER, ingredient_id INTEGER, ingredient_count INTEGER)''')

conn.commit()

Напишем функцию для добавления блюд:

In [26]:
# Cell #25
dishes = {
    'Панкейки': {
        'Молоко': 210,
        'Яйцо': 1,
        'Мука': 200,
        'Разрыхлитель': 5,
        'Масло растительное': 2,
        'Сахар': 30,
        'Соль': 1
    },
    
    'Буррито': {
        'Тортилья': 4,
        'Куриное филе': 12,
        'Фасоль': 4,
        'Кукуруза': 4,
        'Помидор': 8,
        'Перец': 1,
        'Сыр': 200,
        'Лук': 4,
        'Чеснок': 3,
        'Соль': 2
    },
    
    'Овощи по деревенски': {
        'Картофель': 16,
        'Кабачок': 4,
        'Баклажан': 4,
        'Сельдерей': 1,
        'Помидор': 4,
        'Перец': 8,
        'Петрушка': 2
    }
}

In [27]:
# Cell #26
def getInsertId(cursor, table, name):
    cursor.execute("SELECT id FROM '%s' WHERE name=?" % table, (name, ))
    results = cursor.fetchall()
    if(len(results) > 0):
        return results[0][0]
    else:
        cursor.execute("INSERT INTO '%s' (name) VALUES (?)" % table, (name, ))
        return getInsertId(cursor, table, name)

In [28]:
# Cell #27
for dish_name, ingredients in dishes.items():
    for ingredient_name, count in ingredients.items():
        print(ingredient_name, ': ', count, sep='')

Молоко: 210
Яйцо: 1
Мука: 200
Разрыхлитель: 5
Масло растительное: 2
Сахар: 30
Соль: 1
Тортилья: 4
Куриное филе: 12
Фасоль: 4
Кукуруза: 4
Помидор: 8
Перец: 1
Сыр: 200
Лук: 4
Чеснок: 3
Соль: 2
Картофель: 16
Кабачок: 4
Баклажан: 4
Сельдерей: 1
Помидор: 4
Перец: 8
Петрушка: 2


In [29]:
# Cell #28
for dish_name, ingredients in dishes.items():
    for ingredient_name, count in ingredients.items():
        ingredient_id = getInsertId(c, 'ingredients', ingredient_name)
        dish_id = getInsertId(c, 'dishes', dish_name)
        
        c.execute("INSERT INTO dishes_ingredients (dish_id, ingredient_id, ingredient_count) VALUES (?, ?, ?)", 
                       (dish_id, ingredient_id, count))


In [30]:
# Cell #29
c.execute("""SELECT dishes.name, ingredients.name, ingredient_count FROM dishes_ingredients 
          JOIN dishes ON dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id""")
results = c.fetchall()
results

[('Панкейки', 'Молоко', 210),
 ('Панкейки', 'Яйцо', 1),
 ('Панкейки', 'Мука', 200),
 ('Панкейки', 'Разрыхлитель', 5),
 ('Панкейки', 'Масло растительное', 2),
 ('Панкейки', 'Сахар', 30),
 ('Панкейки', 'Соль', 1),
 ('Буррито', 'Тортилья', 4),
 ('Буррито', 'Куриное филе', 12),
 ('Буррито', 'Фасоль', 4),
 ('Буррито', 'Кукуруза', 4),
 ('Буррито', 'Помидор', 8),
 ('Буррито', 'Перец', 1),
 ('Буррито', 'Сыр', 200),
 ('Буррито', 'Лук', 4),
 ('Буррито', 'Чеснок', 3),
 ('Буррито', 'Соль', 2),
 ('Овощи по деревенски', 'Картофель', 16),
 ('Овощи по деревенски', 'Кабачок', 4),
 ('Овощи по деревенски', 'Баклажан', 4),
 ('Овощи по деревенски', 'Сельдерей', 1),
 ('Овощи по деревенски', 'Помидор', 4),
 ('Овощи по деревенски', 'Перец', 8),
 ('Овощи по деревенски', 'Петрушка', 2)]

In [31]:
# Cell #30
# Создаем таблицу для плана блюд
c.execute('''CREATE TABLE dish_plan
             (dish_id INTEGER, count INTEGER)''')

c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (1, 30)")
c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (2, 50)")
c.execute("INSERT INTO dish_plan (dish_id, count) VALUES (3, 70)")

conn.commit()

In [32]:
# Cell #31
c.execute("""SELECT dishes.name, ingredients.name, ingredient_count, dish_plan.count,
            ingredient_count*dish_plan.count FROM dishes_ingredients 
          JOIN dishes ON dishes_ingredients.dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id
          JOIN dish_plan ON dish_plan.dish_id = dishes.id""")
results = c.fetchall()
results

[('Панкейки', 'Молоко', 210, 30, 6300),
 ('Панкейки', 'Яйцо', 1, 30, 30),
 ('Панкейки', 'Мука', 200, 30, 6000),
 ('Панкейки', 'Разрыхлитель', 5, 30, 150),
 ('Панкейки', 'Масло растительное', 2, 30, 60),
 ('Панкейки', 'Сахар', 30, 30, 900),
 ('Панкейки', 'Соль', 1, 30, 30),
 ('Буррито', 'Тортилья', 4, 50, 200),
 ('Буррито', 'Куриное филе', 12, 50, 600),
 ('Буррито', 'Фасоль', 4, 50, 200),
 ('Буррито', 'Кукуруза', 4, 50, 200),
 ('Буррито', 'Помидор', 8, 50, 400),
 ('Буррито', 'Перец', 1, 50, 50),
 ('Буррито', 'Сыр', 200, 50, 10000),
 ('Буррито', 'Лук', 4, 50, 200),
 ('Буррито', 'Чеснок', 3, 50, 150),
 ('Буррито', 'Соль', 2, 50, 100),
 ('Овощи по деревенски', 'Картофель', 16, 70, 1120),
 ('Овощи по деревенски', 'Кабачок', 4, 70, 280),
 ('Овощи по деревенски', 'Баклажан', 4, 70, 280),
 ('Овощи по деревенски', 'Сельдерей', 1, 70, 70),
 ('Овощи по деревенски', 'Помидор', 4, 70, 280),
 ('Овощи по деревенски', 'Перец', 8, 70, 560),
 ('Овощи по деревенски', 'Петрушка', 2, 70, 140)]

In [33]:
# Cell #32
c.execute("""SELECT ingredients.name, SUM(ingredient_count*dish_plan.count) FROM dishes_ingredients 
          JOIN dishes ON dishes_ingredients.dish_id = dishes.id
          JOIN ingredients ON ingredient_id = ingredients.id
          JOIN dish_plan ON dish_plan.dish_id = dishes.id
          GROUP BY ingredients.name""")
results = c.fetchall()
results

[('Баклажан', 280),
 ('Кабачок', 280),
 ('Картофель', 1120),
 ('Кукуруза', 200),
 ('Куриное филе', 600),
 ('Лук', 200),
 ('Масло растительное', 60),
 ('Молоко', 6300),
 ('Мука', 6000),
 ('Перец', 610),
 ('Петрушка', 140),
 ('Помидор', 680),
 ('Разрыхлитель', 150),
 ('Сахар', 900),
 ('Сельдерей', 70),
 ('Соль', 130),
 ('Сыр', 10000),
 ('Тортилья', 200),
 ('Фасоль', 200),
 ('Чеснок', 150),
 ('Яйцо', 30)]

In [34]:
# Cell #33
c.execute("SELECT id FROM ingredients")
results = c.fetchall()
ids = list(map(lambda it: it[0], results))
for i in range(1, 28):
    count = random.randint(10, 100)
    date = datetime.datetime(2019, 11, i, hour=random.randint(12, 18), minute=random.randint(0, 59), second=0, microsecond=0, tzinfo=None)
    ingredient = random.choice(ids)
    c.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", (ingredient, count, date.timestamp()))

In [35]:
# Cell #34
def inWarehouse(cursor):
    cursor.execute("""SELECT name, SUM(count) FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name""")
    return cursor.fetchall()
    
inWarehouse(c)

[('Картофель', 56),
 ('Кукуруза', 248),
 ('Лук', 117),
 ('Молоко', 161),
 ('Мука', 159),
 ('Петрушка', 163),
 ('Сахар', 128),
 ('Соль', 293),
 ('Сыр', 72),
 ('Фасоль', 126)]

In [36]:
# Cell #35
c.execute(""" 
        SELECT name_plan, count_warehouse, count_plan, count_warehouse-count_plan FROM
        (SELECT name as name_warehouse, SUM(count) as count_warehouse FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name)

        JOIN (
            SELECT ingredients.name as name_plan, 
            SUM(ingredient_count*dish_plan.count) as count_plan
            FROM dishes_ingredients 
            JOIN dishes ON dishes_ingredients.dish_id = dishes.id
            JOIN ingredients ON ingredient_id = ingredients.id
            JOIN dish_plan ON dish_plan.dish_id = dishes.id
            GROUP BY ingredients.name
        ) ON name_warehouse = name_plan """)
results = c.fetchall()
results

[('Картофель', 56, 1120, -1064),
 ('Кукуруза', 248, 200, 48),
 ('Лук', 117, 200, -83),
 ('Молоко', 161, 6300, -6139),
 ('Мука', 159, 6000, -5841),
 ('Петрушка', 163, 140, 23),
 ('Сахар', 128, 900, -772),
 ('Соль', 293, 130, 163),
 ('Сыр', 72, 10000, -9928),
 ('Фасоль', 126, 200, -74)]

In [37]:
# Cell #36
supply(c, 'Морепродукты', 10, datetime.datetime.now())
prettyWarehouse(c)

[('Мука', 46, 1572623820.0),
 ('Молоко', 91, 1572699600.0),
 ('Соль', 32, 1572775080.0),
 ('Петрушка', 93, 1572875880.0),
 ('Лук', 52, 1572963120.0),
 ('Молоко', 70, 1573032300.0),
 ('Кукуруза', 46, 1573128480.0),
 ('Мука', 14, 1573226040.0),
 ('Кукуруза', 51, 1573309320.0),
 ('Сыр', 72, 1573392600.0),
 ('Кукуруза', 53, 1573485060.0),
 ('Соль', 97, 1573554300.0),
 ('Фасоль', 74, 1573644360.0),
 ('Сахар', 72, 1573726260.0),
 ('Соль', 78, 1573826160.0),
 ('Кукуруза', 98, 1573902420.0),
 ('Фасоль', 52, 1573998780.0),
 ('Соль', 11, 1574069340.0),
 ('Петрушка', 25, 1574154060.0),
 ('Соль', 15, 1574253840.0),
 ('Сахар', 15, 1574348520.0),
 ('Соль', 60, 1574428680.0),
 ('Мука', 99, 1574520660.0),
 ('Сахар', 41, 1574597220.0),
 ('Картофель', 56, 1574695380.0),
 ('Лук', 65, 1574769480.0),
 ('Петрушка', 45, 1574860980.0),
 ('Морепродукты', 10, 1583761970.674404)]

In [38]:
# Cell #37
c.execute("""SELECT name FROM warehouse JOIN ingredients ON warehouse.ingredient_id = ingredients.id
            
            EXCEPT
            
            SELECT ingredients.name FROM dishes_ingredients 
            JOIN dishes ON dish_id = dishes.id
            JOIN ingredients ON ingredient_id = ingredients.id
            
        """)
results = c.fetchall()
results

[('Морепродукты',)]

## Подсказки к домашнему заданию

In [39]:
# Cell #38
c.execute("SELECT * FROM warehouse ORDER BY date")
results = c.fetchall()
results

[(30, 3, 46, 1572623820.0),
 (31, 1, 91, 1572699600.0),
 (32, 7, 32, 1572775080.0),
 (33, 21, 93, 1572875880.0),
 (34, 15, 52, 1572963120.0),
 (35, 1, 70, 1573032300.0),
 (36, 11, 46, 1573128480.0),
 (37, 3, 14, 1573226040.0),
 (38, 11, 51, 1573309320.0),
 (39, 14, 72, 1573392600.0),
 (40, 11, 53, 1573485060.0),
 (41, 7, 97, 1573554300.0),
 (42, 10, 74, 1573644360.0),
 (43, 6, 72, 1573726260.0),
 (44, 7, 78, 1573826160.0),
 (45, 11, 98, 1573902420.0),
 (46, 10, 52, 1573998780.0),
 (47, 7, 11, 1574069340.0),
 (48, 21, 25, 1574154060.0),
 (49, 7, 15, 1574253840.0),
 (50, 6, 15, 1574348520.0),
 (51, 7, 60, 1574428680.0),
 (52, 3, 99, 1574520660.0),
 (53, 6, 41, 1574597220.0),
 (54, 17, 56, 1574695380.0),
 (55, 15, 65, 1574769480.0),
 (56, 21, 45, 1574860980.0),
 (57, 22, 10, 1583761970.674404)]

In [40]:
# Cell #39
c.execute("""SELECT name, count, date FROM warehouse 
          JOIN ingredients ON warehouse.ingredient_id = ingredients.id
          ORDER BY date""")
results = c.fetchall()
list(map(lambda it: (it[0], it[1], datetime.datetime.fromtimestamp(it[2]).strftime("%Y-%m-%d %H:%M:%S"))
         , results))

[('Мука', 46, '2019-11-01 18:57:00'),
 ('Молоко', 91, '2019-11-02 16:00:00'),
 ('Соль', 32, '2019-11-03 12:58:00'),
 ('Петрушка', 93, '2019-11-04 16:58:00'),
 ('Лук', 52, '2019-11-05 17:12:00'),
 ('Молоко', 70, '2019-11-06 12:25:00'),
 ('Кукуруза', 46, '2019-11-07 15:08:00'),
 ('Мука', 14, '2019-11-08 18:14:00'),
 ('Кукуруза', 51, '2019-11-09 17:22:00'),
 ('Сыр', 72, '2019-11-10 16:30:00'),
 ('Кукуруза', 53, '2019-11-11 18:11:00'),
 ('Соль', 97, '2019-11-12 13:25:00'),
 ('Фасоль', 74, '2019-11-13 14:26:00'),
 ('Сахар', 72, '2019-11-14 13:11:00'),
 ('Соль', 78, '2019-11-15 16:56:00'),
 ('Кукуруза', 98, '2019-11-16 14:07:00'),
 ('Фасоль', 52, '2019-11-17 16:53:00'),
 ('Соль', 11, '2019-11-18 12:29:00'),
 ('Петрушка', 25, '2019-11-19 12:01:00'),
 ('Соль', 15, '2019-11-20 15:44:00'),
 ('Сахар', 15, '2019-11-21 18:02:00'),
 ('Соль', 60, '2019-11-22 16:18:00'),
 ('Мука', 99, '2019-11-23 17:51:00'),
 ('Сахар', 41, '2019-11-24 15:07:00'),
 ('Картофель', 56, '2019-11-25 18:23:00'),
 ('Лук', 65,

In [41]:
# Cell #40 
inWarehouse(c)

[('Картофель', 56),
 ('Кукуруза', 248),
 ('Лук', 117),
 ('Молоко', 161),
 ('Морепродукты', 10),
 ('Мука', 159),
 ('Петрушка', 163),
 ('Сахар', 128),
 ('Соль', 293),
 ('Сыр', 72),
 ('Фасоль', 126)]

In [42]:
# Cell #41
c.execute("""SELECT warehouse.id, count, name FROM warehouse 
          JOIN ingredients ON ingredients.id = warehouse.ingredient_id""")
results = c.fetchall()
results

[(30, 46, 'Мука'),
 (31, 91, 'Молоко'),
 (32, 32, 'Соль'),
 (33, 93, 'Петрушка'),
 (34, 52, 'Лук'),
 (35, 70, 'Молоко'),
 (36, 46, 'Кукуруза'),
 (37, 14, 'Мука'),
 (38, 51, 'Кукуруза'),
 (39, 72, 'Сыр'),
 (40, 53, 'Кукуруза'),
 (41, 97, 'Соль'),
 (42, 74, 'Фасоль'),
 (43, 72, 'Сахар'),
 (44, 78, 'Соль'),
 (45, 98, 'Кукуруза'),
 (46, 52, 'Фасоль'),
 (47, 11, 'Соль'),
 (48, 25, 'Петрушка'),
 (49, 15, 'Соль'),
 (50, 15, 'Сахар'),
 (51, 60, 'Соль'),
 (52, 99, 'Мука'),
 (53, 41, 'Сахар'),
 (54, 56, 'Картофель'),
 (55, 65, 'Лук'),
 (56, 45, 'Петрушка'),
 (57, 10, 'Морепродукты')]

In [43]:
# Cell #42
c.execute("UPDATE warehouse SET count = 5 WHERE id=31")
prettyWarehouse(c)

[('Мука', 46, 1572623820.0),
 ('Молоко', 5, 1572699600.0),
 ('Соль', 32, 1572775080.0),
 ('Петрушка', 93, 1572875880.0),
 ('Лук', 52, 1572963120.0),
 ('Молоко', 70, 1573032300.0),
 ('Кукуруза', 46, 1573128480.0),
 ('Мука', 14, 1573226040.0),
 ('Кукуруза', 51, 1573309320.0),
 ('Сыр', 72, 1573392600.0),
 ('Кукуруза', 53, 1573485060.0),
 ('Соль', 97, 1573554300.0),
 ('Фасоль', 74, 1573644360.0),
 ('Сахар', 72, 1573726260.0),
 ('Соль', 78, 1573826160.0),
 ('Кукуруза', 98, 1573902420.0),
 ('Фасоль', 52, 1573998780.0),
 ('Соль', 11, 1574069340.0),
 ('Петрушка', 25, 1574154060.0),
 ('Соль', 15, 1574253840.0),
 ('Сахар', 15, 1574348520.0),
 ('Соль', 60, 1574428680.0),
 ('Мука', 99, 1574520660.0),
 ('Сахар', 41, 1574597220.0),
 ('Картофель', 56, 1574695380.0),
 ('Лук', 65, 1574769480.0),
 ('Петрушка', 45, 1574860980.0),
 ('Морепродукты', 10, 1583761970.674404)]

In [44]:
# Cell #43
inWarehouse(c)

[('Картофель', 56),
 ('Кукуруза', 248),
 ('Лук', 117),
 ('Молоко', 75),
 ('Морепродукты', 10),
 ('Мука', 159),
 ('Петрушка', 163),
 ('Сахар', 128),
 ('Соль', 293),
 ('Сыр', 72),
 ('Фасоль', 126)]

In [45]:
# Cell #44

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

1. Создать таблицу меню, с указанием цен для каждого блюда
2. Создать таблицу чеков, с указанием какие блюда были проданы
3. Создать функцию, которая бы принимала на вход заказ блюд и создавала в бд соответствующий чек и убирала ингредиенты на складе (которые использовались при приготовлении этих блюд)
4. Создать функцию для подсчета выручки (за день/месяц/за всю жизнь)
5. По желанию - продумать архитектуру, чтобы можно было считать прибыль